In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
config = {
    'project': 'qalf',
    'domain': 'tacred',
    'relation': 'per_title',
    'splits': [0],
    'supervision': 'majority',
#     'learn_deps': False,
#     'gen_model_search_space': 1,
#     'gen_init_params': {
#         'lf_propensity'         : True,
#         'class_prior'           : True,
#         'lf_class_propensity'   : True,
#         'seed'                  : None,
#     },
#     'gen_params_default': {
#         'step_size'     : 0.0001,
#         'decay'         : 0.90,
#         'reg_param'     : 0.50,
#         'epochs'        : 25,
#     },    
    'disc_model_class': 'logreg',    
    'disc_model_search_space': 1,
}

In [3]:
# Get DB connection string and add to globals
# NOTE: $SNORKELDB must be set before any snorkel imports
import os

default_db_name = 'qalf_' + config['domain'] + ('_debug' if config.get('debug', False) else '')
DB_NAME = config.get('db_name', default_db_name)
if 'postgres' in config and config['postgres']:
    DB_TYPE = 'postgres'
else:
    DB_TYPE = 'sqlite'
    DB_NAME += '.db'
DB_ADDR = "localhost:{0}".format(config['db_port']) if 'db_port' in config else ""
os.environ['SNORKELDB'] = '{0}://{1}/{2}'.format(DB_TYPE, DB_ADDR, DB_NAME)
print("$SNORKELDB = {0}".format(os.environ['SNORKELDB']))

$SNORKELDB = sqlite:///qalf_tacred.db


In [4]:
from snorkel import SnorkelSession
session = SnorkelSession()

# Resolve config conflicts (nb_config > local_config > global_config)
from snorkel.contrib.babble.pipelines import merge_configs, get_local_pipeline
config = merge_configs(config)

from snorkel.models import candidate_subclass
candidate_class = candidate_subclass(config['candidate_name'], config['candidate_entities'])

pipeline = get_local_pipeline(config['domain'], config['project'])
pipe = pipeline(session, candidate_class, config)

Overwriting domain=None to domain=tacred
Overwriting splits=[0, 1, 2] to splits=[0]
Overwriting disc_model_search_space=10 to disc_model_search_space=1
Overwriting disc_model_class=lstm to disc_model_class=logreg
Overwriting supervision=generative to supervision=majority
Overwriting project=babble to project=qalf
Using TacredQalfPipeline object.


In [5]:
%time pipe.parse()

Clearing existing...
Running UDF...
[========================================] 100%

Parsed 5200 Documents
Parsed 5200 Sentences
CPU times: user 8.98 s, sys: 850 ms, total: 9.83 s
Wall time: 9.35 s


In [6]:
%time pipe.extract()

Clearing existing...
Running UDF...
[========================================] 100%

Split 0: Extracted 5200 Candidates
CPU times: user 14.1 s, sys: 193 ms, total: 14.3 s
Wall time: 14.2 s


In [7]:
%time pipe.load_gold()

[========================================] 100%

AnnotatorLabels created: 5200
CPU times: user 1min 2s, sys: 241 ms, total: 1min 2s
Wall time: 1min 2s


In [8]:
# %time pipe.featurize()

In [9]:
%time pipe.collect()

*QalfPipeline objects load QA result matrices in the label() method.
CPU times: user 186 µs, sys: 131 µs, total: 317 µs
Wall time: 186 µs


In [10]:
%time pipe.label()

Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 13.8 s, sys: 247 ms, total: 14.1 s
Wall time: 14 s


In [11]:
from snorkel.annotations import load_gold_labels
L_gold_train = load_gold_labels(session, annotator_name='gold', split=0)
# L_gold_dev   = load_gold_labels(session, annotator_name='gold', split=1)
# L_gold_test  = load_gold_labels(session, annotator_name='gold', split=2)

In [14]:
pipe.L_train.lf_stats(session, labels=L_gold_train)
# pipe.L_dev.lf_stats(session, labels=L_gold_dev)
# L_test.lf_stats(session, labels=L_gold_test)

,j,Coverage,Overlaps,Conflicts,TP,FP,FN,TN,Empirical Acc.
What was ENT:subj's title? (0.80 -> +1),0,0.190385,0.167500,0.002885,851,139,0,0,0.859596
What was ENT:subj's job? (0.80 -> +1),1,0.252308,0.177885,0.010192,1079,233,0,0,0.822409
What was the name of the ENT:obj? (0.80 -> +1),2,0.182308,0.171538,0.007308,838,110,0,0,0.883966
Who was the ENT:obj? (0.80 -> +1),3,0.174231,0.164808,0.009038,799,107,0,0,0.881898
What was ENT:subj's title? (<0.01 -> -1),4,0.179038,0.148269,0.002885,0,0,199,732,0.786251
What was ENT:subj's job? (<0.01 -> -1),5,0.129231,0.117308,0.009038,0,0,125,547,0.813988
What was the name of the ENT:obj? (<0.01 -> -1),6,0.293654,0.264615,0.008654,0,0,304,1223,0.800917
Who was the ENT:obj? (<0.01 -> -1),7,0.269808,0.253269,0.006538,0,0,276,1127,0.803279


In [15]:
# %time pipe.supervise()

In [16]:
# %time pipe.classify()